#### FMM(Forward Maximum Match)

In [1]:
def FMM_func(user_dict, sentence):
    """
    正向最大匹配（FMM）
    :param user_dict: 词典
    :param sentence: 句子
    """
    # 词典中最长词长度
    max_len = max([len(item) for item in user_dict])
    start = 0
    while start != len(sentence):
        index = start+max_len
        if index>len(sentence):
            index = len(sentence)
        for i in range(max_len):
            if (sentence[start:index] in user_dict) or (len(sentence[start:index])==1):
                print(sentence[start:index], end='/')
                start = index
                break
            index += -1

In [2]:
user_dict = ['我们', '在', '在野', '生动', '野生', '动物园', '野生动物园', '物','园','玩']
sentence = '我们在野生动物园玩'
sentence_segmented = FMM_func(user_dict, sentence)
print(sentence_segmented)

我们/在野/生动/物/园/玩/None


#### BMM(Backforward Maximum Match)

In [3]:
def BMM_func(user_dict, sentence):
    """
    反向最大匹配（BMM）
    :param user_dict:词典
    :param sentence:句子
    """
    # 词典中最长词长度
    max_len = max([len(item) for item in user_dict])
    result = []
    start = len(sentence)
    while start != 0:
        index = start - max_len
        if index < 0:
            index = 0
        for i in range(max_len):
            if (sentence[index:start] in user_dict) or (len(sentence[index:start])==1):
                result.append(sentence[index:start])
                start = index
                break
            index += 1
    print(result)
    for i in result[::-1]:
        print(i, end='/')

In [4]:
user_dict = ['在', '在野', '生动', '野生', '动物园', '野生动物园', '物','园','玩']
sentence = '我们在野生动物园玩'
sentence_segmented = BMM_func(user_dict, sentence)
print(sentence_segmented)

['玩', '野生动物园', '在', '们', '我']
我/们/在/野生动物园/玩/None


#### 练习题：

- （1）实现双向最大匹配分词方法，
- （2）从人民日报分词语料（199801.txt）中读取所有的词，把这些词作为一个字典，
- （3） 将人民日报分词语料去除每一行的编号，还原成未分词的原始语句，
- （4）根据用户字典，使用双向最大匹配分词方法实现对人民日报语料的分词，并分析实际分词效果。

In [5]:
#### 双向最大匹配分词方法实现
class BiMM():
    def __init__(self, user_dict):
        self.user_dict = user_dict
    def cut(self, sentence):
        """
        正向最大匹配（FMM）
        :param user_dict: 词典
        :param sentence: 句子
        """
        # 词典中最长词长度
        max_len = max([len(item) for item in self.user_dict])
        start = 0
        f_single_word_num = 0 #单字词数量计数
        f_segs = [] #存储分词结果
        while start != len(sentence):
            index = start+max_len
            if index>len(sentence):
                index = len(sentence)
            for i in range(max_len):
                if (sentence[start:index] in self.user_dict) or (len(sentence[start:index])==1):
                    f_segs.append(sentence[start:index])
                    if index-start==1:
                        f_single_word_num+=1
                    #print(sentence[start:index], end='/')
                    start = index
                    break
                index += -1
        """
        反向最大匹配（BMM）
        :param user_dict:词典
        :param sentence:句子
        """
        result = [] #暂存分词结果
        b_segs = [] #存储分词结果
        b_single_word_num = 0 #单字词数量计数
        start = len(sentence)
        while start != 0:
            index = start - max_len
            if index < 0:
                index = 0
            for i in range(max_len):
                #print(sentence[index:start])
                if (sentence[index:start] in self.user_dict) or (len(sentence[index:start])==1):
                    result.append(sentence[index:start])
                    if start-index == 1:
                        b_single_word_num+=1
                    start = index
                    break
                index += 1
        b_segs = result[::-1]
    
        """
        双向最大匹配
        """
        if len(f_segs)<len(b_segs):
            return f_segs
        elif len(f_segs)>len(b_segs):
            return b_segs
        else:
            if f_single_word_num>b_single_word_num:
                return b_segs
            else:
                return f_segs

In [6]:
#测试双向最大匹配分词方法
user_dict = ['我们','在', '在野', '生动', '野生', '动物园', '野生动物园', '物','园','玩']
sentence = '我们在野生动物园玩'
bimm = BiMM(user_dict)
print(bimm.cut(sentence))

['我们', '在', '野生动物园', '玩']


In [11]:
#从人民日报分词语料中读取分词数据，并构建词典
#将人民日报分词语料还原成原文
ribao_dict = [] #语料中出现的所有词
ribao_dict_sorted = [] #过滤掉低频词后的词典
yuanwen = [] #存储语料原文
punctuation = []  #存储标点符号
c = 0
with open("D:/teaching/text mining-2022/03-分词与词性标注/199801.txt","r",encoding="utf-8") as fr, \
open("D:/teaching/text mining-2022/03-分词与词性标注/199801_source.txt","w",encoding="utf-8") as fw:
    for line in fr:
        #if c > 100:
        #    break
        ls = line.strip().split('  ')
        line_segs = []
        for i in range(1,len(ls)):
            #print(ls[i])
            #if '/' in ls[i]:
            end_idx = ls[i].index('/')
            line_segs.append(ls[i][:end_idx])
            ribao_dict.append(ls[i][:end_idx])
        
            if ls[i].endswith('/w'):
                punctuation.append(ls[i][:end_idx])
        fw.write(''.join(line_segs)+'\n')
        yuanwen.append(''.join(line_segs))
        #c+=1
word_count_dict = {}
for word in ribao_dict:
    if word not in word_count_dict:
        word_count_dict[word]=1
    else:
        word_count_dict[word]+=1
sorted_word_count = sorted(word_count_dict.items(), key = lambda item:item[1], reverse=True)

#保存出现频次大于2的词，存入词典
for k,v in sorted_word_count:
    if v>2:
        ribao_dict_sorted.append(k)
ribao_dict_set = set(ribao_dict)
punctuation = set(punctuation)
print(len(ribao_dict_set),len(punctuation), len(ribao_dict_sorted))

56482 47 22111


In [12]:
#使用新词典测试双向最大匹配分词方法
bimm = BiMM(ribao_dict_sorted)
bimm.cut("迈向新世纪")

['迈向', '新', '世纪']

In [ ]:
import time
from tqdm import tqdm #进度条线上模块
time_start = time.time() #计时
source_text = []
with open("D:/teaching/text mining-2022/03-分词与词性标注/199801_source.txt","r",encoding="utf-8") as fr:
    for line in fr:
        source_text.append(line.strip())

bimm = BiMM(ribao_dict_sorted)
with open("D:/teaching/text mining-2022/03-分词与词性标注/199801_source_segs.txt","w",encoding="utf-8") as fw:
    for t in tqdm(range(len(source_text)),desc = 'segging'):
        text = source_text[t]  
        start_idx = 0
        Bi_segs = []
        flag = True
        for idx in range(len(text)):
            if text[idx] in punctuation: #有标点符号的，先根据标点符号节分句子
                Bi_segs.extend(bimm.cut(text[start_idx:idx+1]))
                start_idx = idx+1
                flag = False
        if flag:
            Bi_segs.extend(bimm.cut(text))
        fw.write('/  '.join(Bi_segs)+'\n')
time_end = time.time()
print(time_end-time_start)

segging:  20%|████████████▎                                                  | 4528/23064 [11:18:49<2:25:32,  2.12it/s]